# Homework Assignment 3

Save your assignment in your **mp248** repository, in **mp248/Assignments/hwk.3.ipynb**.  

The pre-grading script will offer feedback on problems P1, P2, P3 and P4.

Your assignment is due at **11:59pm** on **Friday November 15th**.

----

## P.1 - Crime stats

Construct a dictionary whose keys are the crime types, and whose values are lists (in descending order) of the five dates with the most crimes of that type. Only include crimes where there are **at least** five dates with ten or more occurances of that crime type. If two or more dates have the same number of crime records, sort them so that the earlier date appears first, i.e. Dec 10th 2014 would be before Jan 2nd 2015.  Print your dictionary with the **pprint** command.

**Hint:** There are only 6 crime types that meet the above criterion.

In [1]:
%cd ../../mp248-course

/home/user/mp248-course


In [2]:
#import cdat
from cdat import cDat

Crime database library.

Looking for vicpd.geojson in directory ./Data/
 modify the cdatfile and cdatloc variables in cdat.py appropriately
 if the file can not be found.

Complete records in cDat, incomplete records in bDat.



In [3]:
import collections as co
import pprint as pp
import operator as op
import calendar as cal

In [4]:
pp.pprint([cDat[x] for x in range(1)])

[{'ID': 760527874,
  'address': '1200 Block JOHNSON ST',
  'case_number': (16, 22918),
  'city': 'victoria',
  'coords': (-123.348998693722, 48.4255680718449),
  'cre_dt': datetime.datetime(2016, 5, 26, 9, 46, 43),
  'inc_dt': datetime.datetime(2016, 5, 25, 1, 29),
  'type': ('Disorder', 'CAUSE A DISTURBANCE'),
  'upd_dt': datetime.datetime(2016, 6, 23, 19, 40, 33)}]


In [5]:
types = {c['type'] for c in cDat}
di = dict()
for typ in types:
    ctdict = co.defaultdict(int)
    for c in cDat:
        if c['type'] == typ:
            ctdict[c['inc_dt'].date()] += 1
    sorted_ctdict = sorted(list(ctdict.items()), key = op.itemgetter(1))[-1:-6:-1]
    if sorted_ctdict[-1][1] > 5:
        di[typ] = sorted_ctdict 

In [6]:
pp.pprint(di)

{('Assault', 'ASSAULT-COMMON OR TRESPASS'): [(datetime.date(2011, 10, 29), 9),
                                             (datetime.date(2012, 2, 26), 9),
                                             (datetime.date(2011, 9, 2), 7),
                                             (datetime.date(2011, 5, 25), 7),
                                             (datetime.date(2011, 3, 26), 7)],
 ('Disorder', 'CAUSE A DISTURBANCE'): [(datetime.date(2014, 7, 6), 13),
                                       (datetime.date(2015, 4, 20), 11),
                                       (datetime.date(2015, 4, 18), 10),
                                       (datetime.date(2011, 7, 3), 10),
                                       (datetime.date(2014, 10, 5), 10)],
 ('Liquor', 'LIQUOR-INTOX IN PUBLIC PLACE'): [(datetime.date(2011, 7, 1), 44),
                                              (datetime.date(2013, 7, 1), 33),
                                              (datetime.date(2015, 7, 1), 28),
        

In [7]:
print("length =",len(di))

length = 16


----

## P.2 - Average crime locations.

Write a function that uses [spherical polar coordinates](https://en.wikipedia.org/wiki/Spherical_coordinate_system) to convert the **(longitude,latitude)** coordinate pairs into 3-dimensional Euclidean coordinates.  Compute the average location (in 3-dimensional space) of all the crime types in the **cdat.cDat** variable.  Invert your spherical polar coordinates, creating a dictionary indexed by crime types, returning the **(longitude,latitude)** pair of the average location for that crime type. Print your final dictionary using **pp.pprint**. 

**Note:** if you do not recall how longitude and latitude relate to spherical polar coordinates, read the above Wikipedia article carefully, together with the [longitude-latitude](https://en.wikipedia.org/wiki/Geographic_coordinate_system) page.

In [24]:
import numpy as np
import math as ma

In [32]:
def polar_to_cartesian(long, lat):
    '''
    Convert spherical polar coordinates to 3D Euclidean coordinates
    In this case, r = 1
    '''
    # lab x = rcos(lat)cos(long) vs wiki x = rsin(θ)cos(φ)
    # lab y = rcos(lat)sin(long) vs wiki y = rsin(θ)sin(φ)
    # lab z = rsin(lat) vs wiki z = rcos(θ)
    # θ = lat, φ = long
    # First: Convert degree to radians for numpy
    x = np.sin(ma.radians(lat))*np.cos(ma.radians(long))
    y = np.sin(ma.radians(lat))*np.sin(ma.radians(long))
    z = np.cos(ma.radians(lat))
    return(x, y, z)

def cartesian_to_polar(x_coords, y_coords, z_coords):
    '''
    Convert cartesian coordinates to spherical polar coordinates
    '''
    # θ = lat, φ = long
    r = 1
    lat = ma.degrees(np.arccos(z_coords/r))
    long = ma.degrees(np.arctan(y_coords/x_coords))
    #if long < 0:
    #    long = 180 + long
    return(long, lat)

In [33]:
print("(-123.35_, 48.43_) ->",polar_to_cartesian(-123.348998693722, 48.4255680718449))
print("->",cartesian_to_polar(-0.41125540309882863, -0.6249112431808368, 0.6635924438751551))

(-123.35_, 48.43_) -> (-0.41125540309882863, -0.6249112431808368, 0.6635924438751551)
-> (56.65100130627801, 48.4255680718449)


In [16]:
dict2 = dict()
## For each type of crimes, take (lat, long), convert to Cartesian
## Then take the avg of each x,y,z coords by sum/(#num of crimes)
## Convert back to polar. Return a dictionary of avg polar.
for typ in types:
    sum_x = 0
    sum_y = 0
    sum_z = 0
    total_crime_count = 0
    ctdict = co.defaultdict(int)
    for c in cDat:
        if c['type'] == typ:
            ctdict[c['coords']] += 1
    for i in ctdict.items():
        lat1 = i[0][0]
        long1 = i[0][1]
        crime_count = i[1]
        x, y, z = polar_to_cartesian(lat1, long1)
        #print(i[0])
        #print(x,y,z)
        total_crime_count += crime_count
        sum_x += x
        sum_y += y
        sum_z += z 
    avg_coords = (sum_x/ total_crime_count, sum_y/total_crime_count, sum_z/total_crime_count)
    dict2[typ] = cartesian_to_polar(avg_coords[0], avg_coords[1],avg_coords[2])

In [17]:
pp.pprint(dict2)

{('Alarm', 'FALSE ALARMS'): (56.63330977541551, 63.72855910561595),
 ('Arson', 'ARSON-DISREGARD FOR HUMAN LIFE'): (56.63565703091797,
                                               54.50985805480552),
 ('Arson', 'ARSON-PROPERTY'): (56.6316509822618, 51.59940406949776),
 ('Assault', 'ASSAULT AGAINST POLICE OFFICER'): (56.632795761804225,
                                                 56.77419339232461),
 ('Assault', 'ASSAULT-COMMON OR TRESPASS'): (56.630254253081944,
                                             73.12474744226293),
 ('Assault', 'ASSAULTS-OTHER'): (56.63155049690549, 58.23153330122856),
 ('Assault with Deadly Weapon', 'ASSAULT OT P/O-W/WPN OR CBH'): (56.63364146110361,
                                                                 54.81660656340022),
 ('Assault with Deadly Weapon', 'ASSAULT POLICE-W/WEAPON OR CBH'): (56.63299185129474,
                                                                    50.9239546782655),
 ('Assault with Deadly Weapon', 'ASSAULT-AGGRAV

----

## P.3 - Crime + Weather data

Load the crime and weather data, i.e. the **cdat.py** and **wdat.py** libraries.

Determine the set of days where we have crime data and weather data, specifically.

1. Dates where we have crime reports in our **cDat** variable.
2. Dates with crime reports and the max/min/mean temperature readings for Victoria in **wdat**. 
3. Dates with crime reports and the max/min/mean temperature readings and the Rain/Precipitation/Snow totals.

Let the number of elements of (1) be **CDD**.  The number of elements of (2) as **CDTD**.  The number of elements of (3) as **CDTPD**,
then execute the command below.

`print(CDD, 'crime data days', CDTD, 'crime data + temp days', CDTPD, 'crime data + precipitation + temp days')`


----

## P.4 - Crime + Weather data


1. Find all the days where it was **6°C** warmer than the preceeding 7 days, i.e. the entire preceding week.  Compute the average crime rates for each crime type on these days.  Which crimes were more than twice as common on these days, compared to an average day?

2. Find all the days where it was **6°C** colder than the preceeding 7 days. Compute the average crime rates for each crime type on these days. Which crimes were more than twice as common on these days, compared to an average day?

Use the 'Max Temp (°C)' field for P.4. 

**Grading script** in parts 1 and 2, print your exceptional week weather, one exceptional week per line, in the form:

`2013-05-25 12.2 17.7 12.7 15.0 13.8 14.5 16.6 24.0`

if for example you found that May 25th of 2013 was 24.0c with the weather in the preceding week being the numbers between the date object and the temperature reading on 2013-05-25. 

----

## P.5 - ODEs for polynomials.

In Lecture 8a we developed a differential equation associated to a polynomial whose solutions were either periodic or converged to some of the roots of the polynomial. Specifically, the ODE:

$$\frac{dz}{dt} = iP(z)$$

In this homework assignment we ask you to explore this ODE in more detail.

**(a)** Create a Python function that takes as input a polynomial.  Your function will generate a **basins of attraction plot**, i.e. at each point $z_0$ in the plot you will compute a solution to the above ODE satisfying the initial condition $z(0)=z_0$.  Depending on the behaviour of the solution you will color the point $z_0$ accordingly.  You will choose colours according to each of the roots.  If your solution converges to a root, colour the point $z_0$ accordingly.  Similarly, choose a colour for initial values $z_0$ corresponding to periodic solutions.  And colour the point $z_0$ black if it does neither. 

**(b)**

Include plots for the polynomials:

$$z^3-1$$

$$z^4-1$$

$$z^5-1$$

$$z^5-z-1$$

**(c)** Do your experiments suggest that for any polynomial with real coefficients, the solutions to the above ODE are either periodic or converge to a root?  If so, can you identify which roots are limits of solutions, and which initial values give periodic solutions? 

Note: A polynomial $$c_nz^n + c_{n-1}z^{n-1} + \cdots + c_1 z + c_0$$ is said to have real coefficients if $c_i \in \mathbb R$ for all $i$.